#Samarth Motka
#202311023
#Lab Assignment 1

# Loading libraries and Dataset

In [96]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


In [97]:
user = pd.read_csv('/content/drive/MyDrive/Daiict/sem2/deep learning/lab1/User_Data - User_Data.csv')
diabetes = pd.read_csv('/content/drive/MyDrive/Daiict/sem2/deep learning/lab1/diabetes - diabetes.csv')
startups= pd.read_csv('/content/drive/MyDrive/Daiict/sem2/deep learning/lab1/50_Startups - 50_Startups.csv')



#1. User Dataset

# Checking for null values

In [98]:
user.isnull().sum()


User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [99]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [100]:
user.describe()

,User ID,Age,EstimatedSalary,Purchased
count,4.000000e+02,400.000000,400.000000,400.000000
mean,1.569154e+07,37.655000,69742.500000,0.357500
std,7.165832e+04,10.482877,34096.960282,0.479864
min,1.556669e+07,18.000000,15000.000000,0.000000
25%,1.562676e+07,29.750000,43000.000000,0.000000
50%,1.569434e+07,37.000000,70000.000000,0.000000
75%,1.575036e+07,46.000000,88000.000000,1.000000
max,1.581524e+07,60.000000,150000.000000,1.000000


# Preprocessing

## Removing User Id as it is not useful

In [101]:
user = user.drop('User ID',axis=1)

# One-hot encoding Gender columns

In [102]:
user_ec = pd.get_dummies(user, columns = [ 'Gender'])
user_ec

,Age,EstimatedSalary,Purchased,Gender_Female,Gender_Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1
...,...,...,...,...,...
395,46,41000,1,1,0
396,51,23000,1,0,1
397,50,20000,1,1,0
398,36,33000,0,0,1


#Spliting Dataset into Train, Test and Validation sets

In [103]:
from sklearn.model_selection import train_test_split

X = user_ec.drop('Purchased',axis=1)
y = user_ec['Purchased']

X_train, X_temp,y_train, y_temp = train_test_split(X,y ,random_state=104,test_size=0.25,shuffle=True)
X_val, X_test,y_val, y_test = train_test_split(X_temp,y_temp ,random_state=104,test_size=0.6,shuffle=True)

#Scaleling/Normalizing Data

# Normalizing Data

In [104]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform valiadation data
X_val_norm = norm.transform(X_val)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

# Training and validating multiple regression model

In [105]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_val = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
r2 = r2_score(y_val, y_pred_val)
mae = mean_absolute_error(y_val, y_pred_val)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Absolute Error: 0.281077050820315
Mean Squared Error: 0.1130384748675742
R-squared: 0.5374384660150415


#Hyperparameter tuning using Grid search and testing on Test set

In [106]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


# Create a Ridge regression model
model = Ridge()

# Define the grid of hyperparameters to search
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Create the grid search object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

print(f'Best Model Hyperparameters: {grid_search.best_params_}')


Mean Absolute Error: 0.30120706144150744
Mean Squared Error: 0.12867880754214747
R-squared: 0.405335420857855
Best Model Hyperparameters: {'alpha': 100}


# Training and validating logistic regression model

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred_val = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
conf_matrix = confusion_matrix(y_val, y_pred_val)
classification_rep = classification_report(y_val, y_pred_val)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.575
Confusion Matrix:
 [[23  0]
 [17  0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.57      1.00      0.73        23
           1       0.00      0.00      0.00        17

    accuracy                           0.57        40
   macro avg       0.29      0.50      0.37        40
weighted avg       0.33      0.57      0.42        40



#Hyperparameter tuning using Grid search and testing on Test set

In [108]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Create the logistic regression model
logreg = LogisticRegression()

# Create the GridSearchCV object
grid_search = GridSearchCV(logreg, param_grid, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_

# Get the best logistic regression model
best_logreg = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f'Best Parameters: {best_params}')
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)


Best Parameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.8666666666666667
Confusion Matrix:
 [[39  2]
 [ 6 13]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.95      0.91        41
           1       0.87      0.68      0.76        19

    accuracy                           0.87        60
   macro avg       0.87      0.82      0.84        60
weighted avg       0.87      0.87      0.86        60



#2. Daiabetes Dataset

In [109]:
diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# Checking null values

In [110]:
diabetes.isnull().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [111]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [112]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


#Preprocessing

#Spliting Dataset into Train, Test and Validation sets

In [113]:
from sklearn.model_selection import train_test_split

X = diabetes.drop('Outcome',axis=1)
y = diabetes['Outcome']

X_train, X_temp,y_train, y_temp = train_test_split(X,y ,random_state=104,test_size=0.25,shuffle=True)
X_val, X_test,y_val, y_test = train_test_split(X_temp,y_temp ,random_state=104,test_size=0.6,shuffle=True)

#Scaleing/Normalizing Data

#Normalizing Data

In [114]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform validation data
X_val_norm = norm.transform(X_val)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

# Training and validating a logistic regression model

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred_val = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
conf_matrix = confusion_matrix(y_val, y_pred_val)
classification_rep = classification_report(y_val, y_pred_val)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.7631578947368421
Confusion Matrix:
 [[44  6]
 [12 14]]
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.88      0.83        50
           1       0.70      0.54      0.61        26

    accuracy                           0.76        76
   macro avg       0.74      0.71      0.72        76
weighted avg       0.76      0.76      0.75        76



#Testing the model

In [116]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.8017241379310345
Confusion Matrix:
 [[68 10]
 [13 25]]
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.87      0.86        78
           1       0.71      0.66      0.68        38

    accuracy                           0.80       116
   macro avg       0.78      0.76      0.77       116
weighted avg       0.80      0.80      0.80       116



#3. Startup Dataset

In [117]:
startups

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94
5,131876.90,99814.71,362861.36,New York,156991.12
6,134615.46,147198.87,127716.82,California,156122.51
7,130298.13,145530.06,323876.68,Florida,155752.60
8,120542.52,148718.95,311613.29,New York,152211.77
9,123334.88,108679.17,304981.62,California,149759.96


# Checking null values

In [118]:
startups.isnull().sum()


R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [119]:
startups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [120]:
startups.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


#Preprocessing

# One-hot encoding Gender columns

In [121]:
startups_ec = pd.get_dummies(startups, columns = [ 'State'])
startups_ec

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
0,165349.20,136897.80,471784.10,192261.83,0,0,1
1,162597.70,151377.59,443898.53,191792.06,1,0,0
2,153441.51,101145.55,407934.54,191050.39,0,1,0
3,144372.41,118671.85,383199.62,182901.99,0,0,1
4,142107.34,91391.77,366168.42,166187.94,0,1,0
5,131876.90,99814.71,362861.36,156991.12,0,0,1
6,134615.46,147198.87,127716.82,156122.51,1,0,0
7,130298.13,145530.06,323876.68,155752.60,0,1,0
8,120542.52,148718.95,311613.29,152211.77,0,0,1
9,123334.88,108679.17,304981.62,149759.96,1,0,0


#Spliting Dataset into Train, Test and Validation sets

In [122]:
from sklearn.model_selection import train_test_split

X = startups_ec.drop('Profit',axis=1)
y = startups_ec['Profit']

X_train, X_temp,y_train, y_temp = train_test_split(X,y ,random_state=104,test_size=0.25,shuffle=True)
X_val, X_test,y_val, y_test = train_test_split(X_temp,y_temp ,random_state=104,test_size=0.6,shuffle=True)

#Scaling/Normalizing Data

# Normalizing Data

In [123]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

#Training and Validating Multiple Regression Model

In [124]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred_val = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
r2 = r2_score(y_val, y_pred_val)
mae = mean_absolute_error(y_val, y_pred_val)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Absolute Error: 6652.057781775805
Mean Squared Error: 59670089.217895165
R-squared: 0.7589906988238405


#Testing the model

In [125]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 5732.383967478316
Mean Squared Error: 48186952.43507526
R-squared: 0.9067888308943415


#Trainig and validating on regression model with Lasso regularization

In [126]:
from sklearn.linear_model import Lasso

# Train a Lasso regression model
alpha = 0.1  # L1 regularization strength (adjust as needed)
lasso_reg = Lasso(alpha=alpha)
lasso_reg.fit(X_train, y_train)

# Evaluate the model on the validation set
y_pred_val = lasso_reg.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
r2 = r2_score(y_val, y_pred_val)
mae = mean_absolute_error(y_val, y_pred_val)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Absolute Error: 6651.754802584828
Mean Squared Error: 59667195.378058925
R-squared: 0.7590023871307593


#Testing the model

In [127]:
# testing the model on the test set
y_pred = lasso_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 5732.343448140886
Mean Squared Error: 48186850.042407654
R-squared: 0.9067890289591656
